<a href="https://colab.research.google.com/github/emanuelebrizzi/bootcamp/blob/main/code_vulnerability_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.2 MB/s eta 0:00:00


# **Data upload**


In [3]:
#from google.colab import files
#import zipfile
#import os

from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
import os
import glob

# Directory path
drive_path = "/content/drive/MyDrive/bootcamp_file"

libpng_path = os.path.join(drive_path, "LibPNG")

libpng_vuln_path = os.path.join(libpng_path, "Vulnerable_functions")
libpng_non_vuln_path = os.path.join(libpng_path, "Non_vulnerable_functions")

vlc_path = os.path.join(drive_path, "VLC")

vlc_vuln_path = os.path.join(vlc_path, "Vulnerable_functions")
vlc_non_vuln_path = os.path.join(vlc_path, "Non_vulnerable_functions")

pidgin_path = os.path.join(drive_path, "Pidgin")

pidgin_vuln_path = os.path.join(pidgin_path, "Vulnerable_functions")
pidgin_non_vuln_path = os.path.join(pidgin_path, "Non_vulnerable_functions")


In [5]:

def load_files_from_folder(folder, label):
    files = glob.glob(os.path.join(folder, "*.c"))
    data = []
    for file in files:
        with open(file, "r", encoding="utf-8", errors="ignore") as f:
            data.append((f.read(), label, os.path.basename(file)))
    return data


# **Replace Function names and Variable names**

In [6]:
import re
import random
import string
from pathlib import Path

In [7]:
# Removes comments from C code
def remove_comments(code):
    code = re.sub(r'//.*', '', code)
    code = re.sub(r'/\*.*?\*/', '', code, flags=re.DOTALL)
    return code

# Extracts variable and function names from C code
def extract_identifiers(code):
    pattern = re.compile(r'\b([a-zA-Z_][a-zA-Z0-9_]*)\b')
    keywords = set(['int', 'char', 'float', 'double', 'return', 'if', 'else', 'while', 'for', 'do', 'switch', 'case', 'void'])
    identifiers = set(pattern.findall(code)) - keywords
    return identifiers

# Classifies identifiers as variables or functions
def categorize_identifiers(identifiers):
    var_counter, fun_counter = 1, 1
    replacements = {}
    for identifier in identifiers:
        if re.search(r'\b[A-Za-z_][A-Za-z0-9_]*\s*\(', identifier):
            replacements[identifier] = f'fun_{fun_counter}'
            fun_counter += 1
        else:
            replacements[identifier] = f'var_{var_counter}'
            var_counter += 1
    return replacements

# Replaces variable and function names with new names
def replace_identifiers(code, replacements):

    for old, new in replacements.items():
        code = re.sub(r'\b' + re.escape(old) + r'\b', new, code)
    return code

# Reads a C file, replaces names, and saves the new file
def process_c_file(filepath, new_filepath):
    with open(filepath, 'r', encoding='Latin-1') as file:
        code = file.read()

    code_no_comments = remove_comments(code)
    identifiers = extract_identifiers(code_no_comments)
    replacements = categorize_identifiers(identifiers)
    new_code = replace_identifiers(code_no_comments, replacements)
    with open(new_filepath, 'w', encoding='Latin-1') as file:
        file.write(new_code)

# Processes all C files in a directory
def process_directory(directory, new_directory_path):
    for filepath in Path(directory).glob("*.c"):
        new_filepath = os.path.join(new_directory_path, os.path.basename(filepath))
        process_c_file(filepath, new_filepath)

Cleanning code:

In [8]:
import shutil

def create_directory(directory_path):
    if os.path.exists(directory_path):
        shutil.rmtree(directory_path)
    os.makedirs(directory_path, exist_ok=True)

# Creation modified directory
modified_path = os.path.join(drive_path, "modified_files")
#create_directory(modified_path)

libpng_modified = os.path.join(modified_path, "LibPNG")
#create_directory(libpng_modified)

vlc_modified = os.path.join(modified_path, "VLC")
#create_directory(vlc_modified)

pidgin_modified = os.path.join(modified_path, "Pidgin")
#create_directory(pidgin_modified)

libpng_modified_vuln_path = os.path.join(libpng_modified, "Vulnerable_functions")
#create_directory(libpng_modified_vuln_path)
libpng_modified_non_vuln_path = os.path.join(libpng_modified, "Non_vulnerable_functions")
#create_directory(libpng_modified_non_vuln_path)

vlc_modified_vuln_path = os.path.join(vlc_modified, "Vulnerable_functions")
#create_directory(vlc_modified_vuln_path)
vlc_modified_non_vuln_path = os.path.join(vlc_modified, "Non_vulnerable_functions")
#create_directory(vlc_modified_non_vuln_path)

pidgin_modified_vuln_path = os.path.join(pidgin_modified, "Vulnerable_functions")
#create_directory(pidgin_modified_vuln_path)
pidgin_modified_non_vuln_path = os.path.join(pidgin_modified, "Non_vulnerable_functions")
#create_directory(pidgin_modified_non_vuln_path)

#process_directory(libpng_vuln_path, libpng_modified_vuln_path)
#process_directory(libpng_non_vuln_path, libpng_modified_non_vuln_path)
#process_directory(vlc_vuln_path, vlc_modified_vuln_path)
#process_directory(vlc_non_vuln_path, vlc_modified_non_vuln_path)
#process_directory(pidgin_vuln_path, pidgin_modified_vuln_path)
#process_directory(pidgin_non_vuln_path, pidgin_modified_non_vuln_path)


# Dataset preparation

In [9]:
import os
import numpy as np
from collections import Counter
from tabulate import tabulate
import random
import os
import re

In [8]:
# Load and label data
def load_data(directory, label):
    data = []
    for filename in os.listdir(directory):
        with open(os.path.join(directory, filename), 'r') as file:
            data.append((file.read(), label))
    return data

vulnerable_files = load_data(libpng_modified_vuln_path, label=1)
non_vulnerable_files = load_data(libpng_modified_non_vuln_path, label=0)

# Combine and shuffle dataset
dataset = vulnerable_files + non_vulnerable_files
np.random.shuffle(dataset)

texts, labels = zip(*dataset)

In [ ]:
# Load and label data
def load_data_directoies(directories, label):
    data = []
    for directory in directories:  # Iteriamo sulle directory della lista
        for filename in os.listdir(directory):  # Ora directory è una stringa
            with open(os.path.join(directory, filename), 'r') as file:
                data.append((file.read(), label))
    return data
vulnerable_files = load_data_directoies([libpng_modified_vuln_path, vlc_modified_vuln_path, pidgin_modified_vuln_path], label=1)
non_vulnerable_files = load_data_directoies([libpng_modified_non_vuln_path, vlc_modified_non_vuln_path, pidgin_modified_non_vuln_path], label=0)

# Combine and shuffle dataset
dataset = vulnerable_files + non_vulnerable_files
np.random.shuffle(dataset)

texts, labels = zip(*dataset)

### Dataset study

In [ ]:
print("Vulnerable files: ", len(vulnerable_files))
print("Non Vulnerable files: ", len(non_vulnerable_files))
print("Dataset files: ", len(dataset))

In [ ]:
n = len(vulnerable_files) * 8

# Selezione casuale senza modificare la struttura dei dati
non_vulnerable_files_selected = random.sample(non_vulnerable_files, min(n, len(non_vulnerable_files)))

# Combina e mescola il dataset
dataset = vulnerable_files + non_vulnerable_files_selected
random.shuffle(dataset)

# Dividi in testi e label
texts, labels = zip(*dataset)


In [ ]:
print("Vulnerable files: ", len(vulnerable_files))
print("Non Vulnerable files: ", len(non_vulnerable_files_selected))
print("Dataset files: ", len(dataset))

In [19]:
# Lista per salvare i nomi dei file nella forma CVE-XXXX-XXXX
cve_files = []

# Espressione regolare per trovare i nomi compatibili
pattern = re.compile(r"(?i)(?:^|_)cve-(\d{4})-(\d{4})")

# Scansiona la directory
for file in os.listdir(libpng_modified_vuln_path):
    # Cerca nel nome del file
    match = pattern.search(file)
    if match:
        # Formatta il nome nella forma CVE-XXXX-XXXX
        formatted_name = f"CVE-{match.group(1)}-{match.group(2)}"
        cve_files.append(formatted_name)


for file in os.listdir(vlc_modified_vuln_path):
    # Cerca nel nome del file
    match = pattern.search(file)
    if match:
        # Formatta il nome nella forma CVE-XXXX-XXXX
        formatted_name = f"CVE-{match.group(1)}-{match.group(2)}"
        cve_files.append(formatted_name)


for file in os.listdir(pidgin_modified_vuln_path):
    # Cerca nel nome del file
    match = pattern.search(file)
    if match:
        # Formatta il nome nella forma CVE-XXXX-XXXX
        formatted_name = f"CVE-{match.group(1)}-{match.group(2)}"
        cve_files.append(formatted_name)



print("Vulnerable files: (CVE) ", len(cve_files))


# Conta tutte le occorrenze
occurrences = Counter(cve_files)

# Creiamo una tabella con i dati
table_data = [(cve, count) for cve, count in occurrences.items()]

# Stampa in formato tabella leggibile
print(tabulate(table_data, headers=["CVE", "Occorrenze"], tablefmt="grid"))


Vulnerable files: (CVE)  118
+---------------+--------------+
| CVE           |   Occorrenze |
+===============+==============+
| CVE-2015-0973 |            1 |
+---------------+--------------+
| CVE-2004-0599 |            2 |
+---------------+--------------+
| CVE-2014-9495 |            1 |
+---------------+--------------+
| CVE-2011-3464 |            1 |
+---------------+--------------+
| CVE-2011-0408 |            2 |
+---------------+--------------+
| CVE-2004-0597 |            3 |
+---------------+--------------+
| CVE-2007-2445 |            1 |
+---------------+--------------+
| CVE-2006-0481 |            1 |
+---------------+--------------+
| CVE-2012-3425 |            1 |
+---------------+--------------+
| CVE-2008-5907 |            1 |
+---------------+--------------+
| CVE-2018-1378 |            1 |
+---------------+--------------+
| CVE-2011-3328 |            1 |
+---------------+--------------+
| CVE-2007-5269 |            2 |
+---------------+--------------+
| CVE-2015-812

## CVE Classification

In [ ]:
import requests

def get_cve_details(cve_id):
    # URL to obtain vulnerability details from the NVD in JSON format
    #url = f'https://api.nvd.nist.gov/vuln/detail/{cve_id}'
    #url = f'https://services.nvd.nist.gov/rest/json/{cves}/2.0'
    url = f'https://services.nvd.nist.gov/rest/json/cve/1.0/{CVE_ID}'

    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Error fetching data for {cve_id}: {response.status_code}")
        return None

Classification by CVSS

* **Low**: CVSS score from 0.0 to 3.9
* **Medium**: CVSS score from 4.0 to 6.9
* **High**: CVSS score from 7.0 to 8.9
* **Critical**: CVSS score from 9.0 to 10.0



In [ ]:
def get_cve_cvss(cve_id):
    data = get_cve_details(cve_id)
    if data is None:
        return None
    try:
      # Estrai il punteggio CVSS dal dizionario JSON
      cvss_score = data['result']['CVE_Items'][0]['impact']['baseMetricV2']['score']
      return cvss_score
    except KeyError:
      print(f"CVSS score not found for {cve_id}")
      return None

cvss = []

# Itera sulla lista e ottieni il punteggio CVSS per ogni CVE
for cve in cve_files:
    cvss = get_cve_cvss(cve)
    if cvss is not None:
        print(f"{cve}: CVSS Score = {cvss}")


Classification by CWE

In [ ]:
def get_cve_cwe(cve_id):
  data = get_cve_details(cve_id)
  if data is None:
    return None
  try:
    # Estrai il CWE dal dizionario JSON
    cwe = data['result']['CVE_Items'][0]['cve']['problemtype']['problemtype_data'][0]['description'][0]['value']
    return cwe
  except KeyError:
    print(f"CWE not found for {cve_id}")
    return None

cwe = []

# Itera sulla lista e ottieni il CWE per ogni CVE
for cve in cve_files:
    cwe = get_cve_cwe(cve)
    if cwe is not None:
        print(f"{cve}: CWE = {cwe}")


# **BLSTM**

In [11]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout

## Model

In [12]:
# Tokenize the data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index

Without K-Fold

In [13]:


# Pad sequences
max_length = 500  # Adjust based on dataset
data = pad_sequences(sequences, maxlen=max_length, padding='post')
labels = np.array(labels)

# Split dataset into train, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

# Define BLSTM model
embedding_dim = 128
model = Sequential([
    Embedding(input_dim=len(word_index) + 1, output_dim=embedding_dim, input_length=max_length),
    Bidirectional(LSTM(32, return_sequences=True, dropout=0.3, recurrent_dropout=0.1)),
    Bidirectional(LSTM(16, return_sequences=False, dropout=0.3, recurrent_dropout=0.1)),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model on training data
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate model on test data
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

# Predict on test data
predictions = model.predict(X_test)
y_pred = (predictions > 0.5).astype(int)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 90s 6s/step - accuracy: 0.7279 - loss: 0.6307 - val_accuracy: 0.8706 - val_loss: 0.3997
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 74s 6s/step - accuracy: 0.7820 - loss: 0.4845 - val_accuracy: 0.8706 - val_loss: 0.3677
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 68s 6s/step - accuracy: 0.8062 - loss: 0.4762 - val_accuracy: 0.8706 - val_loss: 0.3682
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 70s 6s/step - accuracy: 0.7997 - loss: 0.4647 - val_accuracy: 0.8588 - val_loss: 0.3680
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 80s 6s/step - accuracy: 0.8389 - loss: 0.4375 - val_accuracy: 0.8588 - val_loss: 0.3686
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 84s 7s/step - accuracy: 0.8711 - loss: 0.3741 - val_accuracy: 0.8588 - val_loss: 0.3599
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 70s 6s/step - accuracy: 0.8266 - loss: 0.4365 - val_accuracy: 0.8588 - val_loss: 0.3398
Epoch 8/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 80s 6s/step - accuracy: 0.8623 - loss: 0.3801 - val_accuracy: 0.8588 - val_loss:

With K-Fold

In [14]:
from sklearn.model_selection import KFold

# Pad sequences
max_length = 500  # Adjust based on dataset
data = pad_sequences(sequences, maxlen=max_length, padding='post')
labels = np.array(labels)

# Define K-Fold Cross-Validation
k_folds = 10
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

embedding_dim = 128
batch_size = 32
epochs = 10

accuracies = []

for fold, (train_index, test_index) in enumerate(kf.split(data)):
    print(f"Training on Fold {fold+1}/{k_folds}...")

    X_train, X_test = data[train_index], data[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    # Creazione del modello
    model = Sequential([
        Embedding(input_dim=len(word_index) + 1, output_dim=embedding_dim, input_length=max_length),
        #Bidirectional(LSTM(32, return_sequences=True, dropout=0.3, recurrent_dropout=0.1)),
        Bidirectional(LSTM(16, return_sequences=False, dropout=0.3, recurrent_dropout=0.1)),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)

    loss, accuracy = model.evaluate(X_test, y_test)
    accuracies.append(accuracy)
    print(f"Fold {fold+1} - Test Loss: {loss}, Test Accuracy: {accuracy}")

print(f"Average Accuracy: {np.mean(accuracies)}")



Training on Fold 1/10...
Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 58s 3s/step - accuracy: 0.8475 - loss: 0.6036 - val_accuracy: 0.8047 - val_loss: 0.4570
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 50s 3s/step - accuracy: 0.8523 - loss: 0.3966 - val_accuracy: 0.8047 - val_loss: 0.4737
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 81s 3s/step - accuracy: 0.8446 - loss: 0.3936 - val_accuracy: 0.8203 - val_loss: 0.4423
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 83s 3s/step - accuracy: 0.8622 - loss: 0.4057 - val_accuracy: 0.8203 - val_loss: 0.4457
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 80s 3s/step - accuracy: 0.8722 - loss: 0.3878 - val_accuracy: 0.8281 - val_loss: 0.4392
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 84s 3s/step - accuracy: 0.8807 - loss: 0.3643 - val_accuracy: 0.8438 - val_loss: 0.4278
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 80s 3s/step - accuracy: 0.8600 - loss: 0.3927 - val_accuracy: 0.8359 - val_loss: 0.4235
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 82s 3s/step - accuracy: 0.8884 - loss: 0.3519 - val_acc

KeyboardInterrupt: 